In [26]:
import pandas as pd
import os
import pickle
from collections import Counter

In [39]:
import pickle
with open("data/topic_dict_topic2FNimg.pkl","rb") as f:
    topic_dict_topic2FNimg = pickle.load(f)

len(topic_dict_topic2FNimg)

272

In [40]:
performance_data = pd.read_csv('data/function_wise.csv')

In [35]:
topic_list = performance_data['topic']

 Aligning topic names

In [11]:
a = set(list(topic_dict_topic2FNimg.keys())) - set(topic_list)

In [9]:
set(topic_list)- set(list(topic_dict_topic2FNimg.keys()))

{'drainage', 'reading light', 'seeing the back of book', 'smells)', 'sofa'}

In [10]:
topic_dict_topic2FNimg['drainage'] = topic_dict_topic2FNimg[' drainage']
topic_dict_topic2FNimg['reading light'] = topic_dict_topic2FNimg[' reading light']
topic_dict_topic2FNimg['seeing the back of book'] = topic_dict_topic2FNimg[' seeing the back of book']
topic_dict_topic2FNimg['smells)'] = topic_dict_topic2FNimg[' smells)']
topic_dict_topic2FNimg['sofa'] = topic_dict_topic2FNimg[' sofa']

In [13]:
for t in a:
    topic_dict_topic2FNimg.pop(t)

In [14]:
len(topic_dict_topic2FNimg)

270

In [31]:
dict_topic2FNimg = topic_dict_topic2FNimg

Calaculating recall for topics

In [21]:
data = pd.read_csv('data/clean_data.csv')

In [22]:
# map each image to corresponding topics, countries, income level ... TODO
dict_img2topic, dict_img2country, dict_img2incomelevel, dict_country2region = {}, {}, {}, {}
for list_topics, image_path, country, incomelevel, region in zip(data['topics'], data['imageRelPath'], data['country.name'], data['quartile'], data['region.id']):
    dict_img2topic[image_path] = list_topics
    dict_img2country[image_path] = country
    dict_img2incomelevel[image_path] = incomelevel
    dict_country2region[country] = region

print(len(dict_img2topic))

36754


In [24]:
# map each topic to list of corresponding images
dict_topic2img = {}
for list_topics, image_path in zip(data['topics'], data['imageRelPath']):
    for topic in list_topics.split(","):
        topic = topic.lower().strip()
        if topic not in dict_topic2img:
            dict_topic2img[topic] = set() #### here a set was used in place of list to avoid duplicate where keyword appears twice in a topic
        dict_topic2img[topic].add(image_path)

print(len(dict_topic2img))

270


In [27]:
list_all_incomelevelsGT = []
dict_topic2GTIncomelevels = {}
dict_all_incomelevels_GT = {}
for topic in dict_topic2img:
    list_imagesGT = dict_topic2img[topic]
    list_incomelevelsGT = [dict_img2incomelevel[img] for img in list_imagesGT]
    for incomelevel in list_incomelevelsGT:### Topics where Poor has higher Recall than other classes (Appendix)
        list_all_incomelevelsGT.append(incomelevel)
    #print(f"topic: {topic}; {Counter(list_countriesFN)}")
    dict_topic2GTIncomelevels[topic] = Counter(list_incomelevelsGT)
    
#print(f"ALL topics: {Counter(list_all_incomelevelsGT)}")
dict_all_incomelevels_GT = Counter(list_all_incomelevelsGT)

In [28]:
dict_topic2GTrichvspoor = {}
for topic, values in dict_topic2GTIncomelevels.items():
    temp_dict = {}
    temp_dict['low'] = values['poor'] + values['lower-middle']
    temp_dict['high'] = values['rich'] + values['upper-middle']
    
    dict_topic2GTrichvspoor[topic] = temp_dict
len(dict_topic2GTrichvspoor)

270

In [32]:
list_all_incomelevelsFN = []
dict_topic2FNIncomelevels = {}
dict_all_incomelevels_FN = {}
for topic in dict_topic2FNimg:
    list_imagesFN = dict_topic2FNimg[topic]
    list_incomelevelsFN = [dict_img2incomelevel[img] for img in list_imagesFN]
    for incomelevel in list_incomelevelsFN:
        list_all_incomelevelsFN.append(incomelevel)
    #print(f"topic: {topic}; {Counter(list_countriesFN)}")
    dict_topic2FNIncomelevels[topic] = Counter(list_incomelevelsFN)
    
#print(f"ALL topics: {Counter(list_all_incomelevelsFN)}")
dict_all_incomelevels_FN = Counter(list_all_incomelevelsFN)

In [33]:
dict_topic2FNrichvspoor = {}
for topic, values in dict_topic2FNIncomelevels.items():
    temp_dict = {}
    temp_dict['low'] = values['poor'] + values['lower-middle']
    temp_dict['high'] = values['rich'] + values['upper-middle']
    
    dict_topic2FNrichvspoor[topic] = temp_dict
len(dict_topic2FNrichvspoor)

270

In [34]:
Recall_perincomelevel_pertopic = {}
for topic in dict_topic2FNrichvspoor:
    FN_counter_ = dict_topic2FNrichvspoor[topic]
    GT_counter_ = dict_topic2GTrichvspoor[topic]
        
    temp_recall_ratio_ = Counter({key : 1 - FN_counter_[key] / GT_counter_[key] for key in FN_counter_ if sum(GT_counter_.values()) * 0.01 <= GT_counter_[key] })
    Recall_perincomelevel_pertopic[topic] = temp_recall_ratio_

In [41]:
t_list = topic_list
topic_high = [round(Recall_perincomelevel_pertopic[topic]['high'], 2) for topic in t_list]
topic_low = [round(Recall_perincomelevel_pertopic[topic]['low'], 2) for topic in t_list]



performance_data['topic_low'] = topic_low
performance_data['topic_high'] = topic_high

In [45]:

performance_data = performance_data.drop(columns = ['Unnamed: 0'])
performance_data.head()

,topic,function_low,function_high,topic_low,topic_high
0,family snapshots,0.12,0.32,0.02,0.27
1,cutlery,0.39,0.52,0.36,0.60
2,family,0.63,0.48,0.23,0.47
3,place where eating dinner,0.03,0.17,0.14,0.47
4,plate of food,0.33,0.47,0.28,0.47


In [46]:
performance_data['gap_topic'] = performance_data['topic_high'] - performance_data['topic_low']
performance_data['gap_function'] = performance_data['function_high'] - performance_data['function_low']
performance_data.head()

,topic,function_low,function_high,topic_low,topic_high,gap_topic,gap_function
0,family snapshots,0.12,0.32,0.02,0.27,0.25,0.20
1,cutlery,0.39,0.52,0.36,0.60,0.24,0.13
2,family,0.63,0.48,0.23,0.47,0.24,-0.15
3,place where eating dinner,0.03,0.17,0.14,0.47,0.33,0.14
4,plate of food,0.33,0.47,0.28,0.47,0.19,0.14


In [47]:
performance_data['delta_gap'] = performance_data['gap_topic'] - performance_data['gap_function']

In [48]:
performance_data.head()

,topic,function_low,function_high,topic_low,topic_high,gap_topic,gap_function,delta_gap
0,family snapshots,0.12,0.32,0.02,0.27,0.25,0.20,0.05
1,cutlery,0.39,0.52,0.36,0.60,0.24,0.13,0.11
2,family,0.63,0.48,0.23,0.47,0.24,-0.15,0.39
3,place where eating dinner,0.03,0.17,0.14,0.47,0.33,0.14,0.19
4,plate of food,0.33,0.47,0.28,0.47,0.19,0.14,0.05


In [49]:
sum(performance_data['delta_gap'])/len(performance_data['delta_gap'])

0.0924444444444445

In [50]:
performance_data['delta_gap'].median()

0.06000000000000001

In [51]:
from scipy.stats import wilcoxon

In [52]:
delta_gap = performance_data['delta_gap']

In [57]:
# Wilcoxon signed-rank
stat, p_value = wilcoxon(delta_gap, alternative='greater')
print(f"Median gap reduction = {delta_gap.median():.3f}, p = {p_value:}")

Median gap reduction = 0.060, p = 1.6232759967121248e-17


In [55]:
from scipy.stats import ranksums

In [56]:
ranksums(list(performance_data['gap_topic']), list(performance_data['gap_function']), alternative='greater')

RanksumsResult(statistic=6.272300919269884, pvalue=1.7787558716582637e-10)